In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import quadprog

In [2]:
class Generate_random_fx_and_train_points:
    def __init__(self,range_from,range_to,N):
        self.range_from = range_from
        self.range_to = range_to
        self.N = N
        self.randomize_fx()
        self.randomize_data()
        
    def randomize_fx(self):
        X = np.random.uniform(self.range_from,self.range_to,2)
        Y = np.random.uniform(self.range_from,self.range_to,2)
        
        fit = np.polyfit(X,Y,1)
        self.target = np.poly1d(fit)
    
    def randomize_data(self):
        plus = False
        minus = False
        while(plus == False or minus == False):
            plus = False
            minus = False
            self.X = np.random.uniform(self.range_from,self.range_to,(self.N,2))
            self.Y = []
            for x in self.X:
                y = self.fx(x[0],x[1])
                if(y==1):
                    plus = True
                elif(y==-1):
                    minus = True
                self.Y.append(y)
        self.X = np.hstack((np.ones((len(self.X),1)),self.X))
        self.X = [np.array(i,ndmin=2).T for i in self.X]
        self.Y = np.reshape(self.Y,(self.N,1))
   
    def fx(self,x, y):
        original_y = self.target(x)
        return 1 if original_y < y else -1      

In [3]:
class PLA:
    def __init__(self,fx,X,Y):
        self.fx = fx
        self.X = X
        self.Y = Y
        self.w = np.zeros((3,1))
        self.train()
        self.test_e_out()
        
    def train(self):
        while(True):
            all_correct = True
            
            low = 0
            high = len(self.X)

            while(low<high):
                idx = np.random.randint(low,high)
                answer = np.sign(np.dot(self.w.T,self.X[idx]))
                if(answer != self.Y[idx]):
                    all_correct = False
                    self.w+=self.Y[idx]*self.X[idx]
                else:
                    self.X[low], self.X[idx] = self.X[idx], self.X[low]
                    self.Y[[low,idx]]=self.Y[[idx,low]]
                    low+=1
            
            if(all_correct):
                break
    
    def test_e_out(self):
        test_size = 1000
        X_test = np.random.uniform(-1,1,(test_size, 2))
        X_test = np.hstack((np.ones((len(X_test),1)), X_test))
        X_test = [np.array(i,ndmin=2).T for i in X_test]
        Y_test = [self.target(i[1],i[2]) for i in X_test]
        Y_test = np.reshape(Y_test,(len(Y_test),1))

        errors=0
        for i in range(len(X_test)):
            answer = np.sign(np.dot(self.w.T,X_test[i]))
            if(answer!=Y_test[i]):
                errors+=1
        self.e_out = errors / test_size
        
        
    def target(self,x,y):
        original_y = self.fx(x)
        return 1 if original_y < y else -1

In [4]:
class Plot_everything:
    def __init__(self,target,pla_w,X,Y):
        self.target = target
        self.pla_w = pla_w
        self.X = X
        self.Y = Y
        self.xs = x = np.linspace(-1,1,100)
        
        self.plot_target()
        self.plot_points()
        self.plot_pla()
        
        plt.legend()
        plt.axis([-1,1,-1,1])
        plt.show()
        
    def plot_target(self):
        plt.plot(self.xs,[self.target(i) for i in self.xs],label='Target')
    
    def plot_points(self):
        for i in range(len(self.X)):
            if(self.Y[i]==1):
                plt.plot(self.X[i][1],self.X[i][2],'g.')
            elif(a.Y[i]==-1):
                plt.plot(self.X[i][1],self.X[i][2],'r.')
                
    def plot_pla(self):
        plt.plot(self.xs,[-self.pla_w[0]/self.pla_w[2] - self.pla_w[1]/self.pla_w[2]*i for i in self.xs],label='PLA')

In [335]:
class Hard_margin_svm:
    def __init__(self,fx,X,Y):
        self.fx = fx
        self.X = X
        self.Y = Y
        
        self.construct_p()
        self.q = np.ones((len(self.Y)))
        
        self.g = -np.identity(len(self.Y))
        self.h = np.zeros((len(self.Y)))
        self.construct_a()
        self.b = np.zeros(2)
        
        self.solve()
        
    def construct_p(self):
        self.p = []
        for i in range(len(self.X)):
            row = []
            for j in range(len(self.X)):
                row.append((self.Y[i] * self.Y[j] * np.dot(self.X[i].T , self.X[j])).tolist()[0])
            self.p.append(row)
        self.p = np.reshape(self.p,(len(self.X),len(self.X)))
        self.p = np.array(self.p)
        
    def construct_a(self):      
        self.a = self.Y.T.tolist()
        self.a.append(np.negative(self.Y.T).tolist()[0])
        self.a = np.reshape(self.a,(2,len(self.Y)))
    
    def solve(self):
        qp_G = self.p
        qp_a = -self.q
        
        qp_C = -np.vstack([ self.a , self.g ]).T
        qp_b = -np.hstack([self.b , self.h])
        meq = self.a.shape[0]
    
        alphas = quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]
        print(alphas)

In [320]:
# a = Generate_random_fx_and_train_points(-1,1,10)
# pla = PLA(a.target,a.X,a.Y)
# # p = Plot_everything(a.target,pla.w,a.X,a.Y)

In [341]:
a = Generate_random_fx_and_train_points(-1,1,5)
print(np.dot(a.X[0].T,a.X[0])*1000)
svm = Hard_margin_svm(a.target,a.X,a.Y)

[[ 1167.09481972]]
[[[1.1670948197214406], [1.2563468541900087], [-0.7684684368410682], [-0.7311961944404931], [-1.0638593869509902]], [[1.2563468541900087], [1.493347619840103], [-0.5454274224889664], [-0.6115409728302489], [-0.9433441090479039]], [[-0.7684684368410682], [-0.5454274224889664], [1.4194878251316025], [1.3487074236147363], [1.0650505184206254]], [[-0.7311961944404931], [-0.6115409728302489], [1.3487074236147363], [1.4381411448480514], [0.8603060054526653]], [[-1.0638593869509902], [-0.9433441090479039], [1.0650505184206254], [0.8603060054526653], [1.2633140283564441]]]
[[ 1.16709482  1.25634685 -0.76846844 -0.73119619 -1.06385939]
 [ 1.25634685  1.49334762 -0.54542742 -0.61154097 -0.94334411]
 [-0.76846844 -0.54542742  1.41948783  1.34870742  1.06505052]
 [-0.73119619 -0.61154097  1.34870742  1.43814114  0.86030601]
 [-1.06385939 -0.94334411  1.06505052  0.86030601  1.26331403]]


ValueError: matrix G is not positive definite